In [6]:
import numpy as np
import pandas as pd
from pprint import pprint

from datasets import Dataset
from setfit import SetFitModel

from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitTrainer

import torch

if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")
    
from data.preprocess_data import combine_json_to_dataframe


CUDA is not available.


In [2]:
# load data to score
recipe_data = combine_json_to_dataframe("../data/recipes_raw.zip")

In [18]:
setfit_trainer = torch.load('../models/setfit-recipe-cls.pt', map_location='cpu')
# For some reason the map_location does not work on a SetFit model
# Thus the below hacky way to get a model trained on a GPU to run on my CPU
setfit_trainer.model = setfit_trainer.model.to('cpu')

In [29]:
docs = [rec for rec in recipe_data.full_text[:140]]

class_predictions = setfit_trainer.model.predict(docs)
class_probas = setfit_trainer.model.predict_proba(docs)

In [ ]:
def get_max_index(tensor, col):
    max_val, max_idx = torch.max(tensor[:, col], 0)
    return max_idx.item()

hardest_recipe = get_max_index(class_probas, 3) #hardest recipe

pprint(docs[hardest_recipe.values])